In [ ]:
# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

# Project import
import artpop

# artpop's matplotlib style
plt.style.use(artpop.mpl_style)

# use this random state for reproducibility
rng = np.random.RandomState(122)

In [ ]:
# Example of initializing a stellar population
# Note that you want enough stars to adequately sample the IMF 
# but not so much that you use a ton of memory!

#This will create the object called 'ssp' which stores the information of 100,000 stars created
#From a Salpeter IMF and aged to 1e9 yrs. Each star has its own mass, luminosity, spectra, temperature, etc
#All all of these properties are calculated given all the inputs, such as age and metalicity.
# While the distribution of masses is given by sampling the IMF.

ssp = artpop.MISTSSP(
    log_age = 9,          # log of age in years 
    feh = 0,             # metallicity [Fe/H]
    phot_system = 'LSST', # photometric system(s)
    num_stars = 1e5,      # number of stars.
    imf = 'salpeter',       # default imf
    random_state = rng,   # random state for reproducibility
)


In [ ]:
# We can make an HR diagram (Luminosity vs Temperature) for our stars

#These select different evolutionary phases

MS = ssp.select_phase('MS') #Main Sequence
RGB = ssp.select_phase('RGB') #Red Giant Branch
AGB = ssp.select_phase('AGB') #Asymptotic Giant Branch

#Plot L vs Temp, coloring each phase separately

plt.plot(ssp.log_Teff[MS], ssp.log_L[MS], marker='o',ls='',
         label='MS', c='tab:blue', mec='k')
plt.plot(ssp.log_Teff[RGB], ssp.log_L[RGB], marker='o',ls='',
         label='RGB + subgiant branch',
         c='tab:green', mec='k')
plt.plot(ssp.log_Teff[AGB], ssp.log_L[AGB], marker='o',ls='',
         label='AGB', c='tab:red', mec='k')

plt.legend(loc='lower left')
plt.gca().invert_xaxis()
plt.minorticks_on()
plt.xlabel(r'$\log(\mathrm{T_{eff}/K})$')
plt.ylabel(r'$\log(\mathrm{L/L_\odot})$');

In [ ]:
# Mass-to-Light Ratio
# Total Mass M / Total Luminosity L

mass_to_light = np.sum(ssp.star_masses) / np.sum(10**ssp.log_L)

print(mass_to_light)

In [ ]:
# We can calculate stellar populations for different stellar ages by creating a "GRID"
# of stellar populations, each sampling the same IMF but with different ages
# each element of ssp_grid is a separate population of a different age
# i.e. ssp_grid[2] will be the population with log(age) = 8

ssp_grid = []
log_age = [6,7,8,9,9.5,10]
for i in range(len(log_age)):
    ssp_grid.append(artpop.MISTSSP(
                    log_age = log_age[i],   # log of age in years, looping through the list above
                    feh = 0,             # metallicity [Fe/H]
                    phot_system = 'LSST', # photometric system(s)
                    num_stars = 1e5,      # number of stars.
                    imf = 'salpeter',       # default imf
                    random_state = rng,   # random state for reproducibility
                    ))


In [ ]:
# Now we can calculate the mass to light ratio vs age

mass_tot = np.zeros(len(ssp_grid)) #create arrays to store our data
lum_tot = np.zeros(len(ssp_grid))
for i in range(len(ssp_grid)): #Loop over our grid
    mass_tot[i] = np.sum(ssp_grid[i].star_masses) #sum of all stellar masses 
    lum_tot[i] = np.sum(10**ssp_grid[i].log_L) #sum of all luminosities

mtol = mass_tot/lum_tot #final step! M/L

plt.plot(log_age, mtol, marker='o', ls='-') #now plot
plt.yscale('log')
plt.ylabel('M/L')
plt.xlabel('log Age [yr]')


In [ ]:
# We can understand a bit why this evolution occurs
# By Plotting total mass and luminosity vs time
# Note that the change in luminosity is driving this evolution!
# It decreases by a factor of more than 1000 over 10 Gyr while
# mass decreases by about 30%

plt.plot(log_age, mass_tot/mass_tot[0], marker='o', ls='-', color='b')
#plt.yscale('log')
plt.ylim(0,1)
plt.ylabel('M(t)/M(t=1e6)', color='b')
plt.xlabel('log Age [yr]')
plt.twinx()
plt.plot(log_age, lum_tot/lum_tot[0], marker='^', ls='-',color='r')
plt.yscale('log')
plt.ylim(1e-4,1)
plt.ylabel('L(t)/L(t=1e6)',color='r')
plt.tick_params(axis='y')

In [ ]:
# Now we can see how the stellar populations evolve for different metalicities 
# and initial mass functions

# Create new time grids for alpha and comapre them with the original Salpeter (alpha=2.35)
# Ignore alpha = 1 and alpha = 2 since these present problems for integration
# Hint: you can copy and paste much of the code above! 
# But now you need to loop over both age AND alpha

log_age = [6,7,8,9,9.5,10]
alpha = [1.5,2.75,3.5,4.5] # here are our different slopes (dN/dM = M^-alpha)
imf_ssp_grids = []
for j in range(len(alpha)):
    imf_ssp_grids.append([])
    for i in range(len(log_age)):
        print('making SSP for alpha =', alpha[j],'and log age', log_age[i])
        imf_ssp_grids[j].append(artpop.MISTSSP(
                    log_age = log_age[i],   # log of age in years, looping through the list above
                    feh = 0,             # metallicity [Fe/H]
                    phot_system = 'LSST', # photometric system(s)
                    num_stars = 1e5,      # number of stars.
                    imf = {'a':-1*alpha[j]},       # new imf slope!
                    random_state = rng,   # random state for reproducibility
                    ))

In [ ]:
# Now all_ssp_grids is a 2D list where imf_ssp_grids[j][i] corresponds to alpha[j] and 
# log_age[i]

print('First Grid (0,0):', imf_ssp_grids[0][0])
print(imf_ssp_grids[0][0].imf)

In [ ]:
# Plot your results and compare
# You should have several different lines, each representing the M/L vs time for a different
# IMF slope alpha. Make sure each line is different and labeled and that your plot has a legend

# Notes on legends in matplotlib:
# Use the keyword "label" when plotting (plt.plot(x, y, label='my label')
# It should be a string (so don't forget the '')
# Then use the command plt.legend() to create a legend with your labels
# Line styles are set with the ls keyword (e.g. plt.plot(x, y, ls='--') will make a dashed line)
# Markers are set with the marker keyword (e.g. marker='D' will make a diamond, '^' will make a triangle)
# A list of line styles: https://matplotlib.org/2.0.2/api/lines_api.html#matplotlib.lines.Line2D.set_linestyle
# A list of marker styles: https://matplotlib.org/2.0.2/api/markers_api.html#module-matplotlib.markers
# Don't forget to label your axes! (plt.ylabel() plt.xlabel())

#Example plotting data for first new imf
mtol_imf = []
for i in range(len(imf_ssp_grids[0])):
    mtol_imf.append(np.sum(imf_ssp_grids[0][i].star_masses)/np.sum(10**imf_ssp_grids[0][i].log_L))
plt.plot(log_age,mtol_imf,marker='o', ls=':', label='alpha='+str(alpha[0]))

mtol_imf = []
for i in range(len(imf_ssp_grids[1])):
    mtol_imf.append(np.sum(imf_ssp_grids[1][i].star_masses)/np.sum(10**imf_ssp_grids[1][i].log_L))
plt.plot(log_age,mtol_imf,marker='o', ls='--', label='alpha='+str(alpha[1]))

mtol_imf = []
for i in range(len(imf_ssp_grids[2])):
    mtol_imf.append(np.sum(imf_ssp_grids[2][i].star_masses)/np.sum(10**imf_ssp_grids[2][i].log_L))
plt.plot(log_age,mtol_imf,marker='^', ls='-.', label='alpha='+str(alpha[2]))

mtol_imf = []
for i in range(len(imf_ssp_grids[3])):
    mtol_imf.append(np.sum(imf_ssp_grids[3][i].star_masses)/np.sum(10**imf_ssp_grids[3][i].log_L))
plt.plot(log_age,mtol_imf,marker='D', ls='-', label='alpha='+str(alpha[3]))

#And finally plot the original!
plt.plot(log_age,mtol,marker='o', ls='-', color='k', label='alpha=2.35')

plt.yscale('log')
plt.legend()
plt.ylabel('M/L')
plt.xlabel('log Age [yr]')



In [ ]:
# Now do the same thing but for differetn values of metalicity [Fe/H] = -2, -1, -0.5, and 0.5
# Again, make a new GRID of simple stellar populations with different metalicities and ages

feh = [-2,-1,-0.5,0.5]
log_age = [6,7,8,9,9.5,10]
metal_ssp_grids = []
for j in range(len(feh)):
    metal_ssp_grids.append([])
    for i in range(len(log_age)):
        print('making SSP for [Fe/h] =', feh[j],'and log age', log_age[i])
        metal_ssp_grids[j].append(artpop.MISTSSP(
                    log_age = log_age[i],   # log of age in years, looping through the list above
                    feh = feh[j],             # metallicity [Fe/H]
                    phot_system = 'LSST', # photometric system(s)
                    num_stars = 1e5,      # number of stars.
                    imf = 'salpeter',       # default imf
                    random_state = rng,   # random state for reproducibility
                    ))


In [ ]:
# Then PLOT M/L vs Age for each [Fe/H] value and examine how metalicity affects the evolution
# Dont forget to compare with our original SSP! (alpha = 2.35, [Fe/H]=0)
# Node that this is all the same as above, but with some new variable names

mtol_Z = []
for i in range(len(metal_ssp_grids[0])):
    mtol_Z.append(np.sum(metal_ssp_grids[0][i].star_masses)/np.sum(10**metal_ssp_grids[0][i].log_L))
plt.plot(log_age,mtol_Z,marker='o', ls=':', label='[Fe/h]='+str(feh[0]))

mtol_Z= []
for i in range(len(metal_ssp_grids[1])):
    mtol_Z.append(np.sum(metal_ssp_grids[1][i].star_masses)/np.sum(10**metal_ssp_grids[1][i].log_L))
plt.plot(log_age,mtol_Z,marker='o', ls='--', label='[Fe/h]='+str(feh[1]))

mtol_Z = []
for i in range(len(metal_ssp_grids[2])):
    mtol_Z.append(np.sum(metal_ssp_grids[2][i].star_masses)/np.sum(10**metal_ssp_grids[2][i].log_L))
plt.plot(log_age,mtol_Z,marker='^', ls='-.', label='[Fe/h]='+str(feh[2]))

mtol_Z = []
for i in range(len(metal_ssp_grids[3])):
    mtol_Z.append(np.sum(metal_ssp_grids[3][i].star_masses)/np.sum(10**metal_ssp_grids[3][i].log_L))
plt.plot(log_age,mtol_Z,marker='D', ls='-', label='[Fe/h]='+str(feh[3]))

#And finally plot the original!
plt.plot(log_age,mtol,marker='o', ls='-', color='k', label='[Fe/h]=0')

plt.yscale('log')
plt.legend()
plt.ylabel('M/L')
plt.xlabel('log Age [yr]')

In [ ]:
# In one of the first examples above, we were able to pick out different evolutionary states for our SSPs


# These are indices that can be used to extract ONLY stars fitting these categories
# For example, below would be the total mass for only RGB stars at age = 1e6 yrs

print("RBG mass, age = 1e6 yr, salpeter IMF:", np.sum(ssp_grid[0].star_masses[RGB]))

# We can calculate the fraction of luminosity from just the RGB and AGB stars at each time
LumFrac_RGBAGB_orig = []


for i in range(len(ssp_grid)):
    MS = ssp_grid[i].select_phase('MS') #Main Sequence
    RGB = ssp_grid[i].select_phase('RGB') #Red Giant Branch
    AGB = ssp_grid[i].select_phase('AGB') #Asymptotic Giant Branch
    LumAGB_orig = np.sum(10**ssp_grid[i].log_L[AGB])
    LumRGB_orig = np.sum(10**ssp_grid[i].log_L[RGB])
    LumFrac_RGBAGB_orig.append((LumRGB_orig+LumAGB_orig)/np.sum(10**ssp_grid[i].log_L))
    
    
plt.plot(log_age,LumFrac_RGBAGB_orig, 'o', ls='-',color='k')
plt.ylabel('AGB+RGB Luminosity Fraction')
plt.xlabel('log Age [yr]')

In [ ]:
# Plot the fraction of luminosity coming from RGB+AGB stars as a function of time for different IMFs



In [ ]:
# Plot the fraction of luminosity coming from RGB+AGB stars as a function of time for different Metalicities

